In [ ]:
function subproblem(π, node)
    if SP_VERSION == "v1"
        SPobj, column = subproblem_v1(π, node)
    elseif SP_VERSION == "v2"
        println("truc")
    end
    
    return SPobj, column        
end

In [ ]:
function subproblem_v1(π, node)
    SP = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
    set_optimizer_attribute(SP, "OutputFlag", 0)
    
    @variable(SP, x[(i,j) in EDGES], Bin)
    @variable(SP, y[i in VERTICES], Bin)
    
    for i in VERTICES
        if length(outneighbors(GRAPH,i)) >= 1
            @constraint(SP, sum(x[(i,j)] for j in outneighbors(GRAPH,i)) == y[i])  
        else
            @constraint(SP, 0 == y[i])  
        end
        if length(inneighbors(GRAPH,i)) >= 1
            @constraint(SP, sum(x[(j,i)] for j in inneighbors(GRAPH,i)) == y[i])  
        else
            @constraint(SP, 0 == y[i])  
        end
    end
    
    @constraint(SP, sum(x[(i,j)] for (i,j) in EDGES) <= L)
    @constraint(SP, sum(x[(i,j)] for (i,j) in EDGES) >= 2) 
    
    @objective(SP, Max, sum(x[(i,j)]*WEIGHTS[(i,j)] for (i,j) in EDGES) - sum(π[i]*y[i] for i in VERTICES))
    
    # Ajout en plus des contraintes de branchement de ce noeud
    for (i,j) in tree[node].setzero
        @constraint(SP, x[(i,j)] == 0)
    end
    for (i,j) in tree[node].setone
        @constraint(SP, x[(i,j)] == 1)
    end
                                    
    # Résolution du sous-problème
    optimize!(SP)

    # Si le sous-problème est résolu à l'optimalité alors on retourne la valeur objective
    #  et la colonne associée à la solution trouvée
    # Sinon, si le problème est irréalisable, on retourne -Inf et []
    if JuMP.termination_status(SP) == MOI.OPTIMAL
        column = Dict{String,Any}() 
        column["vertices"] = collect(JuMP.value.(y))
        column["oneedges"] = [(i,j) for (i,j) in EDGES if JuMP.value(x[(i,j)]) == 1.0]
        return JuMP.objective_value(SP), column
    else
        return -Inf, []
    end
end